In [110]:
import requests
import json
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [ ]:


url = "https://raider.io/api/v1/guilds/profile?access_key=RIORcaTkJx1kUf9ETAAs2o3i8&region=us&realm=Ragnaros&name=Chifrijo%20Raiders&fields=members"

payload = {}
headers = {}

responseGuild = requests.request("GET", url, headers=headers, data=payload)





In [ ]:


datajson = json.loads( responseGuild.text)


In [ ]:

data = datajson


memebers = data['members']


df = pd.DataFrame([
    {
        "Nombre": m['character']['name'],
        "Rank": '',
        "Date": ''
    }
    for m in data['members']
])



In [102]:
def get_rank(name, realm):
    import requests
    import json





    url = f"""https://raider.io/api/v1/characters/profile?access_key=RIORcaTkJx1kUf9ETAAs2o3i8&region=us&realm={realm}&name={name}&fields=mythic_plus_scores_by_season%3Aseason-tww-2"""
    
    payload = {}
    headers = {
    'accept': 'application/json'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    response_json = json.loads(response.text)

    return response_json['mythic_plus_scores_by_season'][0]['scores']['all']

In [55]:
def get_last_active_time(name, realm):
    import requests
    import json





    url = f"""https://raider.io/api/v1/characters/profile?access_key=RIORcaTkJx1kUf9ETAAs2o3i8&region=us&realm={realm}&name={name}&fields=mythic_plus_recent_runs"""

    payload = {}
    headers = {
    'accept': 'application/json'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    response_json = json.loads(response.text)

    return response_json['mythic_plus_recent_runs'][0]['completed_at']

In [103]:
ListFinal = []

In [ ]:
fistDay = datetime.datetime.strptime("2025-05-10", "%Y-%m-%d").strftime("%d/%m/%Y")
lastDay = datetime.datetime.strptime("2025-05-20", "%Y-%m-%d").strftime("%d/%m/%Y")

if responseGuild.status_code == 200:
    json_data = responseGuild.json() 
    usuarios = memebers

    for usuario in usuarios:
        rank = get_rank(usuario['character']['name'], usuario['character']['realm'])
        if rank != 0:
            last_active = get_last_active_time(usuario['character']['name'], usuario['character']['realm'])
            last_active_formated = datetime.datetime.strptime(last_active, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%d/%m/%Y")
            if datetime.datetime.strptime (last_active_formated,"%d/%m/%Y") > datetime.datetime.strptime (fistDay,"%d/%m/%Y") and datetime.datetime.strptime (last_active_formated,"%d/%m/%Y") < datetime.datetime.strptime (lastDay,"%d/%m/%Y"):
                ListFinal.append({
                    'Nombre': usuario['character']['name'],
                    'Rank': rank,
                    'Date': last_active_formated
                })
else:
    print(f"Error {responseGuild.status_code}: {responseGuild.text}")

In [116]:
dfListFinal = pd.DataFrame(ListFinal)


In [117]:
dfListFinal.sort_values(by='Rank', ascending=False, inplace=True)

In [119]:
df_sorted = dfListFinal

In [123]:
df_sorted.to_csv('output.csv', index=False)